<a href="https://colab.research.google.com/github/ACRainieri/pyspark/blob/main/Mini_projeto_DSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q imdb-sqlite
!pip install -q pycountry

     |████████████████████████████████| 10.1 MB 11.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [4]:
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings 

warnings.filterwarnings('ignore')
sns.set_theme(style= 'whitegrid')


In [5]:
%%time
!imdb-sqlite

2022-09-09 13:12:00,883 GET https://datasets.imdbws.com/name.basics.tsv.gz -> downloads/name.basics.tsv.gz
2022-09-09 13:12:01,820 GET https://datasets.imdbws.com/title.basics.tsv.gz -> downloads/title.basics.tsv.gz
2022-09-09 13:12:02,575 GET https://datasets.imdbws.com/title.akas.tsv.gz -> downloads/title.akas.tsv.gz
2022-09-09 13:12:03,901 GET https://datasets.imdbws.com/title.principals.tsv.gz -> downloads/title.principals.tsv.gz
2022-09-09 13:12:05,569 GET https://datasets.imdbws.com/title.episode.tsv.gz -> downloads/title.episode.tsv.gz
2022-09-09 13:12:05,770 GET https://datasets.imdbws.com/title.ratings.tsv.gz -> downloads/title.ratings.tsv.gz
2022-09-09 13:12:05,916 Populating database: imdb.db
2022-09-09 13:12:05,917 Applying schema
2022-09-09 13:12:05,918 Importing file: downloads/name.basics.tsv.gz
2022-09-09 13:12:05,918 Reading number of rows ...
2022-09-09 13:12:12,232 Inserting rows into table: people
100% 11912473/11912473 [02:14<00:00, 88318.87 rows/s]
2022-09-09 13:1

In [6]:
conn = sqlite3.connect('imdb.db')

In [7]:
tabelas = pd.read_sql_query("SELECT NAME as 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

tabelas.head()  
tabelas.Table_Name


0      people
1      titles
2        akas
3        crew
4    episodes
5     ratings
Name: Table_Name, dtype: object

In [8]:
tabelas = tabelas["Table_Name"].values.tolist()
tabelas

['people', 'titles', 'akas', 'crew', 'episodes', 'ratings']

In [9]:
for tabela in tabelas:
  #consulta = "PRAGMA_TABLE_INFO({})".format(tabela)
  consulta = 'PRAGMA table_info(' + tabela +')'
  resultado = pd.read_sql_query(consulta, conn)
  print("Esquema da Tabela: "+ tabela)
  display(resultado)
 

Esquema da Tabela: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


Esquema da Tabela: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


Esquema da Tabela: akas


...

,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,0
1,1,person_id,VARCHAR,0,None,0
2,2,category,VARCHAR,0,None,0
3,3,job,VARCHAR,0,None,0
4,4,characters,VARCHAR,0,None,0


Esquema da Tabela: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


Esquema da Tabela: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


Ajustando um Dataset, agrupando em outros o restante da lista.

In [94]:
consulta1 = '''SELECT type, COUNT(*) as COUNT FROM titles group by type'''
resultado1 = pd.read_sql_query(consulta1, conn)

#display(resultado1)

resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()) * 100

#display(resultado1)

others = {}

others['COUNT'] = resultado1[resultado1['percentual'] < 5] ['percentual'].sum()

others['type'] = 'others'

#display(resultado1)
display(others)

#filtra o dataframe de resultado
resultado1 = resultado1[resultado1['percentual'] > 5]

display(resultado1)

resultado1 = resultado1.append(others, ignore_index=True)
#print(resultado1)


{'COUNT': 8.243403482760055, 'type': 'others'}

,type,COUNT,percentual
0,movie,620719,6.738364
1,short,888276,9.642893
2,tvEpisode,6943363,75.375340


Criar um grafico com apenas 4 categorias